In [1]:
import cv2
import numpy as np
import os
from scipy.optimize import least_squares
from tomlkit import boolean
from tqdm import tqdm
import matplotlib.pyplot as plt

from hw3_defs import get_all_matches, normalize_images, grayscale_images, find_features

In [2]:
class Image_loader():
    def __init__(self, img_dir:str, downscale_factor:float, K):
        # loading the Camera intrinsic parameters K
        # with open(img_dir + '/K.txt') as f:
        #     self.K = np.array(list((map(lambda x:list(map(lambda x:float(x), x.strip().split(' '))),f.read().split('\n')))))
        #     self.image_list = []
        self.K = K
        self.image_list = []
        # Loading the set of images
        for image in sorted(os.listdir(img_dir)):
            if image[-4:].lower() == '.jpg' or image[-4:].lower() == '.png':
                self.image_list.append(img_dir + '/' + image)
        
        self.path = os.getcwd()
        self.factor = downscale_factor
        self.downscale()

    
    def downscale(self) -> None:
        '''
        Downscales the Image intrinsic parameter acc to the downscale factor
        '''
        self.K[0, 0] /= self.factor
        self.K[1, 1] /= self.factor
        self.K[0, 2] /= self.factor
        self.K[1, 2] /= self.factor
    
    def downscale_image(self, image):
        for _ in range(1,int(self.factor / 2) + 1):
            image = cv2.pyrDown(image)
        return image

In [3]:
import sys
from os import path as os_path
file_path = os_path.dirname(os_path.realpath(''))
img_dir = file_path + "/HW4/buddha_images"

# I don't want to downscale images???
downscale_factor = 2.0

# Camera intrinsics
K = np.array([[1600.0, 0.0, 960.0],
              [0.0, 1600.0, 540.0],
              [0.0,0.0,1.0]])

img_obj = Image_loader(img_dir,downscale_factor,K)

imgs = []
for file_path1 in img_obj.image_list: 
    imgs.append(img_obj.downscale_image(cv2.imread(file_path1)))

imgs_norm = normalize_images(imgs)
imgs_gray = grayscale_images(imgs_norm)

In [4]:
# Same Hyperparameter settings as HW3
# sift = cv2.SIFT_create(nfeatures=5000, nOctaveLayers=16, contrastThreshold=0.025, edgeThreshold=10, sigma=1.4)

# kp, des, imgs_sift = find_features(sift, imgs_gray)

feature_matches = get_all_matches(imgs, print_matches=False)


In [5]:

def optimal_reprojection_error(obj_points) -> np.array:
    '''
    calculates of the reprojection error during bundle adjustment
    returns error 
    '''
    transform_matrix = obj_points[0:12].reshape((3,4))
    K = obj_points[12:21].reshape((3,3))
    rest = int(len(obj_points[21:]) * 0.4)
    p = obj_points[21:21 + rest].reshape((2, int(rest/2))).T
    obj_points = obj_points[21 + rest:].reshape((int(len(obj_points[21 + rest:])/3), 3))
    rot_matrix = transform_matrix[:3, :3]
    tran_vector = transform_matrix[:3, 3]
    rot_vector, _ = cv2.Rodrigues(rot_matrix)
    image_points, _ = cv2.projectPoints(obj_points, rot_vector, tran_vector, K, None)
    image_points = image_points[:, 0, :]
    error = [ (p[idx] - image_points[idx])**2 for idx in range(len(p))]
    return np.array(error).ravel()/len(p)

def bundle_adjustment(_3d_point, opt, transform_matrix_new, K, r_error) -> tuple:
    '''
    Bundle adjustment for the image and object points
    returns object points, image points, transformation matrix
    '''
    opt_variables = np.hstack((transform_matrix_new.ravel(), K.ravel()))
    opt_variables = np.hstack((opt_variables, opt.ravel()))
    opt_variables = np.hstack((opt_variables, _3d_point.ravel()))

    values_corrected = least_squares(optimal_reprojection_error, opt_variables, gtol = r_error).x
    K = values_corrected[12:21].reshape((3,3))
    rest = int(len(values_corrected[21:]) * 0.4)
    return values_corrected[21 + rest:].reshape((int(len(values_corrected[21 + rest:])/3), 3)), values_corrected[21:21 + rest].reshape((2, int(rest/2))).T, values_corrected[0:12].reshape((3,4))

def to_ply(path, point_cloud, colors) -> None:
    '''
    Generates the .ply which can be used to open the point cloud
    '''
    out_points = point_cloud.reshape(-1, 3) * 200
    out_colors = colors.reshape(-1, 3)
    print(out_colors.shape, out_points.shape)
    verts = np.hstack([out_points, out_colors])


    mean = np.mean(verts[:, :3], axis=0)
    scaled_verts = verts[:, :3] - mean
    dist = np.sqrt(scaled_verts[:, 0] ** 2 + scaled_verts[:, 1] ** 2 + scaled_verts[:, 2] ** 2)
    indx = np.where(dist < np.mean(dist) + 300)
    verts = verts[indx]
    ply_header = '''ply
        format ascii 1.0
        element vertex %(vert_num)d
        property float x
        property float y
        property float z
        property uchar blue
        property uchar green
        property uchar red
        end_header
        '''
    with open(path + '/HW4/res1/' + 'myPC1.ply', 'w') as f:
        f.write(ply_header % dict(vert_num=len(verts)))
        np.savetxt(f, verts, '%f %f %f %d %d %d')

In [6]:
enable_bundle_adjustment = False

# Find Feature Matches

In [7]:
transform_matrix_0 = np.eye(4)[:3,:]
pose_array = img_obj.K.ravel()

pose_0 = np.matmul(img_obj.K, transform_matrix_0)
pose_1 = np.empty((3, 4)) 
total_points = np.zeros((1, 3))
total_colors = np.zeros((1, 3))

feature_0, feature_1 = feature_matches[0]

# Find Essential Matrix

In [8]:
def get_essential_matrix(points1, points2):
    # Essential matrix
    essential_matrix, em_mask = cv2.findEssentialMat(points1, points2, img_obj.K, method=cv2.RANSAC, prob=0.999, threshold=0.4, mask=None)
    # Only keep inlier points
    points1 = feature_0[em_mask.ravel() == 1]
    points2 = feature_1[em_mask.ravel() == 1]

    return essential_matrix, em_mask, (points1, points2)

In [9]:
essential_matrix, em_mask, (feature_0, feature_1) = get_essential_matrix(feature_0, feature_1)

# Recover Pose
Recover Pose from the essential matrix using cv2.recoverPose.<br>
This method returns triangulated 3D points that pass the cheirality check

In [10]:
def get_pose(essential_matrix, em_mask, pts1, pts2):
    _, rot_matrix, tran_matrix, em_mask = cv2.recoverPose(essential_matrix, pts1, pts2, img_obj.K)

    transformation_matrix = np.hstack((rot_matrix, tran_matrix))

    # Apply Cheirality check to points
    f_pts1 = pts1[em_mask.ravel() > 0]
    f_pts2 = pts2[em_mask.ravel() > 0]

    return transformation_matrix, f_pts1, f_pts2

In [11]:
transformation_matrix_1, feature_0, feature_1 = get_pose(essential_matrix, em_mask, feature_0, feature_1)

# Triangulation

In [12]:
def triangulation(point_2d_1, point_2d_2, projection_matrix_1, projection_matrix_2) -> tuple:
    '''
    Triangulates 3d points from 2d vectors and projection matrices
    returns projection matrix of first camera, projection matrix of second camera, point cloud 
    '''
    pt_cloud = cv2.triangulatePoints(point_2d_1, point_2d_2, projection_matrix_1.T, projection_matrix_2.T)
    return projection_matrix_1.T, projection_matrix_2.T, (pt_cloud / pt_cloud[3])    


In [13]:
pose_1 = np.matmul(img_obj.K, transformation_matrix_1)

feature_0_new, feature_1_new, points_3d = triangulation(pose_0, pose_1, feature_0, feature_1)

# Reprojection Error

In [14]:
def reprojection_error(obj_points, image_points, transform_matrix, K, homogenity) ->tuple:
    '''
    Calculates the reprojection error ie the distance between the projected points and the actual points.
    returns total error, object points
    '''
    rot_matrix = transform_matrix[:3, :3]
    tran_vector = transform_matrix[:3, 3]
    rot_vector, _ = cv2.Rodrigues(rot_matrix)
    if homogenity == 1:
        obj_points = cv2.convertPointsFromHomogeneous(obj_points.T)
    image_points_calc, _ = cv2.projectPoints(obj_points, rot_vector, tran_vector, K, None)
    image_points_calc = np.float32(image_points_calc[:, 0, :])
    total_error = cv2.norm(image_points_calc, np.float32(image_points.T) if homogenity == 1 else np.float32(image_points), cv2.NORM_L2)
    return total_error / len(image_points_calc), obj_points

In [15]:
error, points_3d_new = reprojection_error(points_3d, feature_1_new, transformation_matrix_1, img_obj.K, homogenity = 1)
#ideally error < 1
print("REPROJECTION ERROR: ", error)

REPROJECTION ERROR:  0.006156572540759718


# PnP

In [18]:
def PnP(obj_point, image_point , K, dist_coeff, rot_vector, initial) ->  tuple:
    '''
    Finds an object pose from 3D-2D point correspondences using the RANSAC scheme.
    returns rotational matrix, translational matrix, image points, object points, rotational vector
    '''
    if initial == 1:
        obj_point = obj_point[:, 0 ,:]
        image_point = image_point.T
        rot_vector = rot_vector.T 
    
    _, rot_vector_calc, tran_vector, inlier = cv2.solvePnPRansac(obj_point, image_point, K, dist_coeff, cv2.SOLVEPNP_ITERATIVE)
    # Converts a rotation matrix to a rotation vector or vice versa
    rot_matrix, _ = cv2.Rodrigues(rot_vector_calc)

    if inlier is not None:
        image_point = image_point[inlier[:, 0]]
        obj_point = obj_point[inlier[:, 0]]
        rot_vector = rot_vector[inlier[:, 0]]
    return np.hstack((rot_matrix, tran_vector)), image_point, obj_point, rot_vector

In [20]:
_, feature_1_pnp, points_3d_pnp, _ = PnP(points_3d_new, feature_1_new, img_obj.K, np.zeros((5, 1), dtype=np.float32), feature_0_new, initial=1)

# Loop for all Images

In [21]:
def common_points(image_points_1, image_points_2, image_points_3) -> tuple:
    '''
    Finds the common points between image 1 and 2 , image 2 and 3
    returns common points of image 1-2, common points of image 2-3, mask of common points 1-2 , mask for common points 2-3 
    '''
    cm_points_1 = []
    cm_points_2 = []
    for i in range(image_points_1.shape[0]):
        a = np.where(image_points_2 == image_points_1[i, :])
        if a[0].size != 0:
            cm_points_1.append(i)
            cm_points_2.append(a[0][0])

    mask_array_1 = np.ma.array(image_points_2, mask=False)
    mask_array_1.mask[cm_points_2] = True
    mask_array_1 = mask_array_1.compressed()
    mask_array_1 = mask_array_1.reshape(int(mask_array_1.shape[0] / 2), 2)

    mask_array_2 = np.ma.array(image_points_3, mask=False)
    mask_array_2.mask[cm_points_2] = True
    mask_array_2 = mask_array_2.compressed()
    mask_array_2 = mask_array_2.reshape(int(mask_array_2.shape[0] / 2), 2)
    print(" Shape New Array", mask_array_1.shape, mask_array_2.shape)
    return np.array(cm_points_1), np.array(cm_points_2), mask_array_1, mask_array_2

In [22]:
feature_0_loop = feature_0_new
feature_1_loop = feature_1_pnp
points_3d_loop = points_3d_pnp

total_images = len(img_obj.image_list) - 2 
pose_array = np.hstack((np.hstack((pose_array, pose_0.ravel())), pose_1.ravel()))

threshold = 0.5
for i in tqdm(range(total_images)):
    # Downscale the Image
    # Find and Match Features
    features_cur, features_2 = feature_matches[i+1]
    
    if i != 0:
        # Perform Triangulation to get 3D points
        feature_0_loop, feature_1_loop, points_3d_loop = triangulation(pose_0, pose_1, feature_0_loop, feature_1_loop)
        feature_1_loop = feature_1_loop.T
        points_3d_loop = cv2.convertPointsFromHomogeneous(points_3d_loop.T)
        points_3d_loop = points_3d_loop[:, 0, :]
    
    # Finds common points between images
    cm_points_0, cm_points_1, cm_mask_0, cm_mask_1 = common_points(feature_1_loop, features_cur, features_2)
    cm_points_2 = features_2[cm_points_1]
    cm_points_cur = features_cur[cm_points_1]

    # Performs PnP
    transform_matrix_1, cm_points_2, points_3d_loop, cm_points_cur = PnP(points_3d_loop[cm_points_0], cm_points_2, img_obj.K, np.zeros((5, 1), dtype=np.float32), cm_points_cur, initial = 0)
    pose_2 = np.matmul(img_obj.K, transform_matrix_1)

    error, points_3d_loop = reprojection_error(points_3d_loop, cm_points_2, transform_matrix_1, img_obj.K, homogenity = 0)

    
    cm_mask_0, cm_mask_1, points_3d_loop = triangulation(pose_1, pose_2, cm_mask_0, cm_mask_1)
    error, points_3d_loop = reprojection_error(points_3d_loop, cm_mask_1, transform_matrix_1, img_obj.K, homogenity = 1)
    print("Reprojection Error: ", error)
    pose_array = np.hstack((pose_array, pose_2.ravel()))
    # takes a long time to run
    if enable_bundle_adjustment:
        points_3d_loop, cm_mask_1, transform_matrix_1 = bundle_adjustment(points_3d_loop, cm_mask_1, transform_matrix_1, img_obj.K, threshold)
        pose_2 = np.matmul(img_obj.K, transform_matrix_1)
        error, points_3d_loop = reprojection_error(points_3d_loop, cm_mask_1, transform_matrix_1, img_obj.K, homogenity = 0)
        print("Bundle Adjusted error: ",error)
        total_points = np.vstack((total_points, points_3d_loop))
        points_left = np.array(cm_mask_1, dtype=np.int32)
        color_vector = np.array([image_2[l[1], l[0]] for l in points_left])
        total_colors = np.vstack((total_colors, color_vector))
    else:
        total_points = np.vstack((total_points, points_3d_loop[:, 0, :]))
        points_left = np.array(cm_mask_1, dtype=np.int32)
        color_vector = np.array([imgs[i+2][l[1], l[0]] for l in points_left.T])
        total_colors = np.vstack((total_colors, color_vector)) 



    transform_matrix_0 = np.copy(transform_matrix_1)
    pose_0 = np.copy(pose_1)
    # plt.scatter(i, error)
    # plt.pause(0.05)

    feature_0_loop = np.copy(features_cur)
    feature_1_loop = np.copy(features_2)
    pose_1 = np.copy(pose_2)
    #cv2.imshow(img_obj.image_list[0].split('/')[-2], image_2)
    # if cv2.waitKey(1) & 0xff == ord('q'):
    #     break

print("Printing to .ply file")
print(total_points.shape, total_colors.shape)
to_ply(file_path, total_points, total_colors)
print("Completed Exiting ...")
np.savetxt(file_path + '/HW4/res1/' + 'my1_pose_array.csv', pose_array, delimiter = '\n')


100%|██████████| 20/20 [00:00<00:00, 195.52it/s]

 Shape New Array (230, 2) (230, 2)
Reprojection Error:  4.16679380656944
 Shape New Array (238, 2) (238, 2)
Reprojection Error:  1.5356444757322385
 Shape New Array (307, 2) (307, 2)
Reprojection Error:  4.171738544813878
 Shape New Array (274, 2) (274, 2)
Reprojection Error:  11.054346253274444
 Shape New Array (198, 2) (198, 2)
Reprojection Error:  14.466101224411682
 Shape New Array (262, 2) (262, 2)
Reprojection Error:  32.619487694417685
 Shape New Array (222, 2) (222, 2)
Reprojection Error:  5.4476219984359195
 Shape New Array (258, 2) (258, 2)
Reprojection Error:  10.514970878751674
 Shape New Array (402, 2) (402, 2)
Reprojection Error:  20.66441773776696
 Shape New Array (166, 2) (166, 2)
Reprojection Error:  13.13063453870845
 Shape New Array (163, 2) (163, 2)
Reprojection Error:  0.3892490540841538
 Shape New Array (405, 2) (405, 2)
Reprojection Error:  11.144058807670545
 Shape New Array (305, 2) (305, 2)
Reprojection Error:  7.75690118714027
 Shape New Array (333, 2) (333, 

In [23]:
import open3d as o3d

pcd = o3d.io.read_point_cloud(file_path + '/HW4/res1/myPC1.ply')
o3d.visualization.draw_geometries([pcd],
                                  zoom=0.3412,
                                  front=[0.4257, -0.2125, -0.8795],
                                  lookat=[2.6172, 2.0475, 1.532],
                                  up=[-0.0694, -0.9768, 0.2024])

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


RPly: Aborted by user


[Open3D WARNING] Read PLY failed: unable to read file: /home/meuli/src/eece7150/HW4/res1/myPC1.ply
